<a href="https://colab.research.google.com/github/CGame1/Img_classificaton_guide/blob/main/notebooks/create_hf_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
# from datasets import load_dataset


# dataset = load_dataset("imagefolder", data_dir="", split="all")
# dataset[0]["image"]

# import os
# os.path.basename(dataset[0]['image'].filename)

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from datasets import load_dataset
# name= "Prabhjotschugh/Salish-Sea-Fishes"
# name = "zh-plus/tiny-imagenet"
name = "ILSVRC/imagenet-1k"
name= "nf-whoi/coralnet-small"
# name= "esahit/coral-health-classification"

dataset = load_dataset(name)


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image'],
        num_rows: 1599
    })
})

In [ ]:
from pathlib import Path
import numpy as np
import os
from tqdm import tqdm 

# base_path = Path("../images")
base_path = Path("/content/images")

for i in tqdm(np.arange(600)):
    i = int(i)
    save_path = base_path

    # if 'label' in dataset['train'][i].keys():
    #     save_path = save_path / f'{dataset['train'][i]['label']}'

    if os.path.exists(save_path) == False:
        os.mkdir(save_path)

    dataset['train'][i]["image"].save(f"{save_path}/{str(i)}.jpg")

100%|██████████| 4/4 [00:04<00:00,  1.13s/it]
